Hente og lagre data fra JSON data fra ulike APIer, lagre som CSV og JSON filer.

In [27]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install pandasql

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------- ----- 1.8/2.1 MB 13.3 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 11.6 MB/s eta 0:00:00
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26844 sha256=2d89f00173fa56568b8dbab5aa0fe05725f65dcf968ad64f25d68a9593d5e3ab
  Stored in directory: c:\users\silje\appdata\local\pip\cache\wheels\b4\d0\8c\a6b366870bf041849cd96e03b71641e082f8d6456269b603b7
Successfully built pandasql
Note: you may need to restart the kernel to use updated packages.


In [29]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [30]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()
api_key2 = os.getenv("API_KEY2")
url = os.getenv("DATABASER_URL")

#hente og importere data fra API
#Bruker KEY og env slik at dataene ikke skal blir publisert på GIT

url = "https://api.openweathermap.org/data/2.5/weather?q={Trondheim}&appid={your api key}"


In [ ]:
import json
import requests
from datetime import datetime, timedelta
import pandas as pd
import os
from dotenv import load_dotenv


load_dotenv()
client_id = os.getenv('API_KEY3') #Gjør dette med enn .env fil
endpoint = os.getenv('DATABASE_URL3')

parameters = {
    "sources" : "SN68860", 
    "elements" : "mean(air_temperature P1D), mean(air_pressure_at_sea_level P1D), sum(precipitation_amount P1D), mean(wind_speed P1D)",     
    "referencetime" : "2020-03-10/2025-03-19",
}

r = requests.get(endpoint, params = parameters, auth=(client_id, None))

if r.status_code ==200:
    json_data = r.json()
    print("Data hentet fra Frost API")
else: 
    error_msg = r.json().get("error", {}).get("message", "Ukjent feil")
    error_reason = r.json().get("error", {}).get("reason", "Ingen grunn spesifert")
    raise ValueError(f"API-feil: {r.status_code}\nMelding: {error_msg}\nÅrsak: {error_reason}")


vaerdata = []
for entry in json_data.get("data", []):
    timestamp = entry.get("referenceTime", "Ukjent tidspunkt")
    observations = entry.get("observations", [])
    
    weather_entry = {"Tidspunkt" : timestamp}
    for obs in observations: 
        element = obs.get("elementId", "Ukjent element")
        value = obs.get("value", "Ukjent verdi")
        weather_entry[element] = value

    vaerdata.append(weather_entry)

df = pd.DataFrame(vaerdata)

df.dropna(how = "all", subset=df.columns[1:], inplace=True)

df.to_csv("trondheim_vaerdata_full.csv", index = False, encoding="utf-8")

print("data lagret i trondheim_vaerdata_full.csv")

print(df.head())

print(type(df["mean(air_temperature P1D)"][4]))

  

Data hentet fra Frost API
data lagret i trondheim_vaerdata_full.csv
                  Tidspunkt  mean(air_temperature P1D)  \
0  2020-03-10T00:00:00.000Z                        3.6   
1  2020-03-11T00:00:00.000Z                        4.5   
2  2020-03-12T00:00:00.000Z                        3.9   
3  2020-03-13T00:00:00.000Z                        0.0   
4  2020-03-14T00:00:00.000Z                       -3.8   

   mean(air_pressure_at_sea_level P1D)  sum(precipitation_amount P1D)  \
0                                988.7                            0.4   
1                                980.6                            0.1   
2                                980.5                            0.1   
3                               1005.4                            6.7   
4                               1011.1                            4.5   

   mean(wind_speed P1D)  
0                   3.0  
1                   3.0  
2                   2.0  
3                   3.1  
4             

In [3]:
#Lagrer til en JSON-fil
with open("miljodata.json", "w", encoding = "utf-8") as f:  #åpner og oppretter en fil for skriving, med gitt navn. Hvis filen allerede finnes, vil den bli overskrevet. 
        json.dump(vaerdata, f, indent = 4, ensure_ascii=False )     #Tar et python-objekt og skriver det til en fil i JSON-format. Indent= 4 gir mellomrom for hvert nivå. Spesialtegn blir skrevet ut riktig med ascii

print("Data lagret som miljodata.json")  



Data lagret som miljodata.json


In [4]:
#sjekker hva slags data som er lagret

with open("miljodata.json", "r", encoding = "utf-8") as f:  #åpner filen for lesing, encoding sikrer at alle tegn håndteres korrekt
    data = json.load(f) #leser  JSON-data fra filen f og konverterer den til et python-objekt som lagres i variabelen data

print(json.dumps(data, indent=4)) #skriver ut innholdet i filen på en lesbar måte

[
    {
        "Tidspunkt": "2020-03-10T00:00:00.000Z",
        "mean(air_temperature P1D)": 3.6,
        "mean(air_pressure_at_sea_level P1D)": 988.7,
        "sum(precipitation_amount P1D)": 0.4,
        "mean(wind_speed P1D)": 3
    },
    {
        "Tidspunkt": "2020-03-11T00:00:00.000Z",
        "mean(air_temperature P1D)": 4.5,
        "mean(air_pressure_at_sea_level P1D)": 980.6,
        "sum(precipitation_amount P1D)": 0.1,
        "mean(wind_speed P1D)": 3
    },
    {
        "Tidspunkt": "2020-03-12T00:00:00.000Z",
        "mean(air_temperature P1D)": 3.9,
        "mean(air_pressure_at_sea_level P1D)": 980.5,
        "sum(precipitation_amount P1D)": 0.1,
        "mean(wind_speed P1D)": 2
    },
    {
        "Tidspunkt": "2020-03-13T00:00:00.000Z",
        "mean(air_temperature P1D)": 0,
        "mean(air_pressure_at_sea_level P1D)": 1005.4,
        "sum(precipitation_amount P1D)": 6.7,
        "mean(wind_speed P1D)": 3.1
    },
    {
        "Tidspunkt": "2020-03-14T00:00:


import json
import requests
from datetime import datetime, timedelta

lat = 63.4308 #breddegrad Trondheim
lon = 10.4034 #lengdegrad Trondheim
start_date = "2025-03-09"
end_date = "2025-03-16"




url1= f"https://api.met.no/weatherapi/climatehistory/v1/daily?lat={lat}&lon={lon}&start={start_date}&end={end_date}"#api med lat= breddegrad og lon = lengdegrad i Trondheim
HEADERS = {"User-Agent": "Prosjektet/2.0 (siljnord@stud.ntnu.no)"}  
response = requests.get(url1, headers= HEADERS)

if response.status_code == 200:
    data = response.json()

else: 
    print(f"feil ved henting av data: {response.status_code}")
    data = None

#Henter data fra en uke tilbake i tid
if data: 
    #liste som skal inneholde værdata for den siste uken
    trondheim_vaerdata = []

    #beregner dato for en uke tilbake

    one_week_ago = datetime.utcnow() - timedelta(days = 7)


    for forecast in data.get("properties",{}).get("timeseries", []):
        tidspunkt = forecast.get("time", "ukjent_tidspunkt")

        #konverterer tidspunkt til en datetime
        tidspunkt_dt = datetime.strptime(tidspunkt, "%Y-%m-%dT%H:%M:%SZ")

        #sjekker at tidspunktet er innenfor den siste uken
        if tidspunkt_dt >= one_week_ago:
            instant_data = forecast.get("data", {}).get("instant", {}).get("details", {})

            temperatur = instant_data.get("air_temperature", "Data mangler")
            vindhastighet = instant_data.get("wind_speed", "Data mangler")
            
            nedbor = forecast.get("data", {}).get("next_1_hours", {}).get("details", {}).get("precipitation_amount",0)

        #legg til data i listen
        trondheim_vaerdata.append({
            "tidspunkt": tidspunkt, 
            "temperatur": temperatur, 
            "vindhastighet": vindhastighet,
            "nedbor": nedbor
        })

        #lagrer værdata til fil 
        with open ("trondheim_vaerdata.json", "w", encoding = "utf-8") as f:
            json.dump(trondheim_vaerdata, f, indent= 4, ensure_ascii= False)

        print("Data lagret som trondheim_vaerdata.json")


        #skriv ut data
        with open ("trondheim_vaerdata.json", "r", encoding = "utf-8") as f:
            data = json.load(f)

        print(json.dumps(data, indent=4, ensure_ascii = False))



OPPGAVE 3

In [5]:
import pandas as pd 


#identifisere og håndtere databehandling

#identifisere verdier som mangler 
#sjekker hvor mange verdier som mangler i hver kolonne
df = pd.read_csv("trondheim_vaerdata_full.csv")
print(df.isnull().sum())

#fylle inn manglende verdier basert på type data
#numeriske verdier, mean() og median() for å fylle ut hullene


df.fillna({
    "mean(air_temperature P1D)": df["mean(air_temperature P1D)"].median(), 
    "mean(air_pressure_at_sea_level P1D)": df["mean(air_pressure_at_sea_level P1D)"].median(), 
    "sum(precipitation_amount P1D)": df["sum(precipitation_amount P1D)"].fillna(0), #null nedbor
    "mean(wind_speed P1D)": df["mean(wind_speed P1D)"]. fillna(df["mean(wind_speed P1D)"].mean())
}, inplace =True)


#interpolerer for å finne ut manglende verdier
df.interpolate(method = 'linear', inplace= True)


Tidspunkt                              0
mean(air_temperature P1D)              0
mean(air_pressure_at_sea_level P1D)    0
sum(precipitation_amount P1D)          0
mean(wind_speed P1D)                   0
dtype: int64


C:\Users\silje\AppData\Local\Temp\ipykernel_28988\126067788.py:24: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df.interpolate(method = 'linear', inplace= True)


In [6]:
#Bruker list comprehensions for effektiv datamanipilering

#kategorisering av temperatur
df["TemperaurKategori"] = ["Kaldt" if temp < 10
                            else "Mildt" if temp < 20 
                            else "Vamrt" for temp in df["mean(air_temperature P1D)"]]

#kategorisering av vind
df["VindKategori"] = ["Vindstille" if vind < 2
                      else "Lett bris" if vind < 10
                      else "Sterk vind" if vind < 20
                      else "storm"for vind in df["mean(wind_speed P1D)"]]


#Kategorisering av trykk
df["TrykkKategori"] = ["Høytrykk" if trykk > 1013
                       else "Lavtrykk" for trykk in df["mean(air_pressure_at_sea_level P1D)"]]

#Kategorisering av nedbor
df["NedborKategori"] = ["Tørt" if nedbor == 0
                        else "Lett regn" if nedbor < 5
                        else "Mye regn " if nedbor < 20
                        else "Ekstrem vær" for nedbor in df["sum(precipitation_amount P1D)"]]

df.head()  #skriver ut de fem øverste



,Tidspunkt,mean(air_temperature P1D),mean(air_pressure_at_sea_level P1D),sum(precipitation_amount P1D),mean(wind_speed P1D),TemperaurKategori,VindKategori,TrykkKategori,NedborKategori
0,2020-03-10T00:00:00.000Z,3.6,988.7,0.4,3.0,Kaldt,Lett bris,Lavtrykk,Lett regn
1,2020-03-11T00:00:00.000Z,4.5,980.6,0.1,3.0,Kaldt,Lett bris,Lavtrykk,Lett regn
2,2020-03-12T00:00:00.000Z,3.9,980.5,0.1,2.0,Kaldt,Lett bris,Lavtrykk,Lett regn
3,2020-03-13T00:00:00.000Z,0.0,1005.4,6.7,3.1,Kaldt,Lett bris,Lavtrykk,Mye regn
4,2020-03-14T00:00:00.000Z,-3.8,1011.1,4.5,3.9,Kaldt,Lett bris,Lavtrykk,Lett regn


In [7]:
#Manipulerer dataene effektivt med pandas sql

from pandasql import sqldf

query = """ 
SELECT Tidspunkt, "mean(air_temperature P1D)", "sum(precipitation_amount P1D)", "mean(wind_speed P1D)"
FROM df 
WHERE "mean(air_temperature P1D)" > 20 AND "sum(precipitation_amount P1D)"==0 AND "mean(wind_speed P1D)" < 5
ORDER BY "mean(air_temperature P1D)", "sum(precipitation_amount P1D)",  "mean(wind_speed P1D)"DESC
"""

df_warm_days = sqldf(query, locals())  # Finner den varmeste dagen
print(df_warm_days)


                   Tidspunkt  mean(air_temperature P1D)  \
0   2024-07-21T00:00:00.000Z                       20.2   
1   2020-06-25T00:00:00.000Z                       20.5   
2   2023-06-21T00:00:00.000Z                       20.6   
3   2023-08-20T00:00:00.000Z                       20.8   
4   2020-06-17T00:00:00.000Z                       21.0   
5   2024-05-24T00:00:00.000Z                       21.5   
6   2024-05-23T00:00:00.000Z                       21.6   
7   2024-05-26T00:00:00.000Z                       21.6   
8   2020-06-18T00:00:00.000Z                       21.6   
9   2023-06-20T00:00:00.000Z                       21.6   
10  2022-06-26T00:00:00.000Z                       21.9   
11  2023-06-19T00:00:00.000Z                       22.0   
12  2024-08-07T00:00:00.000Z                       22.2   
13  2022-07-01T00:00:00.000Z                       22.3   
14  2020-06-27T00:00:00.000Z                       22.5   
15  2022-06-27T00:00:00.000Z                       23.1 

In [8]:
#HÅNDTERING AV UREGELMESSIGHETER
#værdata som kan være feilmålinger, som ekstreme temperaturer osv, plutselig hopp
df = df[(df["mean(air_temperature P1D)"]>-50) & (df["mean(air_temperature P1D)"]<50)]
df = df[(df["sum(precipitation_amount P1D)"]>900) & (df["sum(precipitation_amount P1D)"]<1100)]
df = df[(df["mean(wind_speed P1D)"]>45) & (df["mean(wind_speed P1D)"]<0)]

#erstatt feil med NaN
df.replace(-9999, None, inplace=True)
df["TempDiff"] = df["mean(air_temperature P1D)"].diff() #beregn differansen mellom hver måling
df["UnormalHoppTemp"] = df["TempDiff"].apply(lambda x: "Ja" if abs(x) > 10 else "Nei") #Markerer plutselige endringer

df["PressureDiff"] = df["mean(air_pressure_at_sea_level P1D)"].diff() #differanse mellom hver måling
df["UnormalHoppPressure"] = df["PressureDiff"].apply(lambda x: "Ja" if abs(x) > 100 else "Nei") #Markerer plutselige endringer


df["Wind_speedDiff"] = df["mean(wind_speed P1D)"].diff()  #differanse mellom hver måling
df["UnormalHoppWind"] = df["Wind_speedDiff"].apply(lambda x: "Ja" if abs(x) > 10 else "Nei") #Markerer plutselige endringer

df["NedborDiff"] = df["sum(precipitation_amount P1D)"].diff()  #differanse mellom hver måling
df["UnormalHoppNedbor"] = df["NedborDiff"].apply(lambda x: "Ja" if abs(x) > 10 else "Nei") #Markerer plutselige endringer



DEL 2 

In [ ]:
#OPPGAVE 4 DATAANALYSE

import numpy as np

gjennomsnitt_temp = np.mean(df["mean(air_temperature P1D)"])
median_temp = np.median(df["mean(air_temperature P1D)"])
std_temp = np.std(df["mean(air_temperature P1D)"])      #avvik fra gjennomsnitt/ hvor spredt dataene er


gjennomsnitt_pressure = np.mean(df["mean(air_pressure_at_sea_level P1D)"])
median_pressure = np.median(df["mean(air_pressure_at_sea_level P1D)"])
std_pressure = np.std(df["mean(air_pressure_at_sea_level P1D)"])

gjennomsnitt_wind = np.mean(df["mean(wind_speed P1D)"])
median_wind = np.median(df["mean(wind_speed P1D)"])
std_wind = np.std(df["mean(wind_speed P1D)"])

gjennomsnitt_nedbor = np.mean(df["sum(precipitation_amount P1D)"])
median_nedbor = np.median(df["sum(precipitation_amount P1D)"])
std_nedbor = np.std(df["sum(precipitation_amount P1D)"])

print(f"Gjennomsnittlig temperatur: {gjennomsnitt_temp: .2f}°C") #printer med 2 desimaler
print(f"Median temperatur: {median_temp: .2f}°C")
print(f"Standardavvik temperatur {std_temp: .2f}°C")

print(f"Gjennomsnittlig trykk: {gjennomsnitt_pressure: .2f} hPa")
print(f"Median trykk: {median_pressure: .2f}hPa")
print(f"Standardavvik trykk {std_pressure: .2f}hPa")

print(f"Gjennomsnittlig vind: {gjennomsnitt_wind: .2f}m/s")
print(f"Median vind: {median_wind: .2f}m/s")
print(f"Standardavvik vind {std_wind: .2f}°m/s")

print(f"Gjennomsnittlig nedbør: {gjennomsnitt_nedbor: .2f}mm")
print(f"Median nedbør: {median_nedbor: .2f}mm")
print(f"Standardavvik nedbør {std_nedbor: .2f}mm")

print(df.columns)

Gjennomsnittlig temperatur:  nan°C
Median temperatur:  nan°C
Standardavvik temperatur  nan°C
Gjennomsnittlig trykk:  nan hPa
Median trykk:  nanhPa
Standardavvik trykk  nanhPa
Gjennomsnittlig vind:  nanm/s
Median vind:  nanm/s
Standardavvik vind  nan°m/s
Gjennomsnittlig nedbør:  nanmm
Median nedbør:  nanmm
Standardavvik nedbør  nanmm
Index(['Tidspunkt', 'mean(air_temperature P1D)',
       'mean(air_pressure_at_sea_level P1D)', 'sum(precipitation_amount P1D)',
       'mean(wind_speed P1D)', 'TemperaurKategori', 'VindKategori',
       'TrykkKategori', 'NedborKategori', 'TempDiff', 'UnormalHoppTemp',
       'PressureDiff', 'UnormalHoppPressure', 'Wind_speedDiff',
       'UnormalHoppWind', 'NedborDiff', 'UnormalHoppNedbor'],
      dtype='object')


In [ ]:
#implementerer statistisk analyse for sammenheng mellom to variabler i datasettet
#Sjekker sammenheng mellom temperatur og lufttrykk ved å beregne korrelasjonskoeffessient
#Sjekker dette for å se om høytrykk og høy temperatur har en sammenheng

korrelasjon = df["mean(air_temperature P1D)"].corr(df["mean(air_pressure_at_sea_level P1D)"])
print(f"Korrelasjon mellom temperatur og trykk: {korrelasjon:.2f}")

Korrelasjon mellom temperatur og trykk: nan


In [ ]:
#Visualiserer sammenhengen mellom temperatur og trykk

import matplotlib.pyplot as plt

plt.scatter(df["mean(air_temperature P1D)"], df["mean(air_pressure_at_sea_level P1D)"])
plt.xlabel("Temperatur (°C)")
plt.ylabel("Lufttrykk (hPa)")
plt.title("Sammenheng mellom temperatur og lufttemperatur")
plt.show()

In [ ]:
#Histogram for temperaturfordeling
plt.hist(["air_temperature"], bins = 20, edgecolor = "blue")
plt.xlabel("Temperatur (°C)")
plt.ylabel("Histogram over temperaturmålinger")
plt.show()

In [ ]:
#linjeplott for trykkvariasjoner per dag
df["Dag"] = pd.to_datetime(df["Tidspunkt"])
df.sort_values("Tidspunkt", inplace = True)

plt.plot(df["Tidspunkt"], df["mean(air_pressure_at_sea_level P1D)"])
plt.xlabel("Dato")
plt.ylabel("Lufttrykk (hPa)")
plt.title("Trykkvariasjoner pr dag")
plt.xticks(rotation = 45)
plt.show()

In [ ]:
#scatterplot for temperatur vs vind
plt.scatter(df["mean(air_temperature P1D)"], df["mean(wind_speed P1D)"])
plt.xlabel("Temperatur (°C)")
plt.ylabel("Vindhastighet (m/s)")
plt.title("Sammenheng mellom temperatur og vind")
plt.show()